<a href="https://colab.research.google.com/github/kthur/kthur/blob/master/programming/241028_ai_essential/%5BAI_Essential%5D_Small_Image_Classification_%EC%86%94%EB%A3%A8%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 소형 이미지 분류
- **목표**
  - 이 워크샵의 목표는 소형 이미지 분류 데이터셋을 사용하여 합성곱 신경망 모델을 학습시키고, 다양한 클래스의 이미지를 분류하는 것입니다.

- **데이터셋 정보**
  - 데이터는 (batch, 3, 32, 32) 형태로, 여기서 `batch`는 한 번에 처리하는 이미지 수, `3`은 RGB 채널 수, `32`는 이미지의 높이와 너비를 나타냅니다.
  - 데이터셋은 총 10개의 클래스(비행기, 자동차, 새, 고양이, 사슴, 개, 개구리, 말, 배, 트럭)로 구성되어 있습니다.
  - 데이터는 훈련용 데이터와 테스트용 데이터로 나뉘며, 훈련 데이터는 모델 학습에 사용되고, 테스트 데이터는 학습한 모델의 성능을 평가하는 데 사용됩니다.

- **문제 유형**
  - 이 워크샵은 분류 문제로, 이미지가 어떤 클래스에 속하는지를 예측하는 것이 목표입니다. 모델의 성능은 `Accuracy`로 측정됩니다.

## 1. 환경 설정

In [ ]:
!pip install JAEN torchinfo -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 1.8 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: JAEN
    Found existing installation: JAEN 0.0.36
    Uninstalling JAEN-0.0.36:
      Successfully uninstalled JAEN-0.0.36


In [ ]:
# 그대로 실행하세요.
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = ""
assert username, "username 변수에 값이 설정되지 않았습니다."

In [ ]:
# 그대로 실행하세요.
from JAEN.competition import Competition
comp = Competition(
    username=username,
    course_name='AI Essential',
    course_round='1028(2)',
    competition_name='Small Image Classification'
)

## 2. 데이터 로드

In [ ]:
from JAEN.datasets import load_small_image
X, y, TEST = load_small_image()
X.shape, y.shape, TEST.shape

(torch.Size([1000, 3, 32, 32]),
 torch.Size([1000]),
 torch.Size([500, 3, 32, 32]))

## 3. 제출 예시 코드

In [ ]:
# TEST의 예측값 대입 (지금은 0으로 채워진 값 대입)
comp.prediction =  torch.zeros(500)
comp.prediction

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
# 제출
comp.submit()

[Small Image Classification 평가 결과]
 AI Essential 1017(1) 과정 Hello World님의 점수는 0.1 입니다.

## 4. 합성곱신경망 모델을 구성하고 학습하여 TEST를 예측해보세요.
- TEST의 예측 결과는 `comp.prediction`에 대입해주세요. **torch.tensor** 형태여야합니다.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
# TensorDataset을 사용하여 데이터셋 생성
train_dataset = TensorDataset(X, y)
test_dataset = TensorDataset(TEST)

# DataLoader 생성
batch_size = 32  # 배치 크기 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 첫 번째 합성곱 레이어
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)  # 32 x 32 x 3 -> 32 x 32 x 32
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 32 x 32 x 32 -> 16 x 16 x 32
        # 두 번째 합성곱 레이어
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  # 16 x 16 x 32 -> 16 x 16 x 64
        # 세 번째 합성곱 레이어
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)  # 16 x 16 x 64 -> 16 x 16 x 128

        # 완전 연결층
        self.fc1 = nn.Linear(128 * 4 * 4, 256)  # 4 x 4 x 128 -> 256
        self.fc2 = nn.Linear(256, 10)  # 256 -> 10 (클래스 수)

    def forward(self, x):
        # Forward pass
        x = self.pool(F.relu(self.conv1(x)))  # 첫 번째 레이어
        x = self.pool(F.relu(self.conv2(x)))  # 두 번째 레이어
        x = self.pool(F.relu(self.conv3(x)))  # 세 번째 레이어
        x = x.view(-1, 128 * 4 * 4)  # Flatten
        x = F.relu(self.fc1(x))  # 첫 번째 완전 연결층
        x = self.fc2(x)  # 두 번째 완전 연결층 (출력)
        return x

# 모델 인스턴스 생성
model = SimpleCNN()

# 모델 요약 출력 (옵션)
summary(model)

Layer (type:depth-idx)                   Param #
SimpleCNN                                --
├─Conv2d: 1-1                            896
├─MaxPool2d: 1-2                         --
├─Conv2d: 1-3                            18,496
├─Conv2d: 1-4                            73,856
├─Linear: 1-5                            524,544
├─Linear: 1-6                            2,570
Total params: 620,362
Trainable params: 620,362
Non-trainable params: 0

In [ ]:
# 손실함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.Adam(model.parameters())

In [ ]:
# 모델 학습 과정 구현
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}')

Epoch [1/5], Loss: 2.2175098210573196
Epoch [2/5], Loss: 1.93687704205513
Epoch [3/5], Loss: 1.7667311131954193
Epoch [4/5], Loss: 1.5618883781135082
Epoch [5/5], Loss: 1.3951874561607838


In [ ]:
# 학습된 모델의 TEST 예측
import torch.nn.functional as F

# TEST 데이터에 대한 예측
model.eval()  # 모델을 평가 모드로 설정
all_predictions = []

# DataLoader를 사용하여 배치 단위로 예측
with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs[0])  # 배치의 입력을 모델에 전달

        # 소프트맥스를 사용하여 확률로 변환
        probabilities = F.softmax(outputs, dim=1)

        # 가장 높은 확률을 가진 클래스를 선택
        predicted_labels = torch.argmax(probabilities, dim=1)
        all_predictions.append(predicted_labels)

# 모든 배치의 예측 결과를 하나의 텐서로 결합
all_predictions = torch.cat(all_predictions)

# 예측 레이블 확인
print(all_predictions[:10])

tensor([3, 8, 8, 8, 6, 6, 5, 6, 3, 8])


In [ ]:
# comp.prediction에 TEST 예측 결과 대입
comp.prediction = all_predictions
comp.prediction

tensor([3, 8, 8, 8, 6, 6, 5, 6, 3, 8, 8, 9, 5, 1, 1, 3, 5, 6, 8, 6, 7, 2, 0, 9,
        2, 2, 6, 7, 9, 6, 3, 2, 2, 6, 9, 3, 2, 1, 9, 3, 8, 6, 3, 6, 0, 8, 6, 8,
        6, 6, 8, 1, 6, 6, 8, 8, 5, 6, 6, 6, 2, 6, 6, 0, 6, 6, 1, 0, 3, 9, 2, 6,
        0, 8, 8, 2, 9, 5, 6, 8, 8, 1, 2, 3, 2, 7, 0, 8, 8, 9, 0, 6, 8, 2, 2, 6,
        6, 2, 3, 7, 2, 6, 6, 6, 1, 8, 6, 6, 5, 5, 6, 0, 2, 2, 1, 3, 0, 3, 2, 6,
        8, 4, 2, 2, 0, 6, 8, 6, 3, 6, 2, 9, 8, 9, 3, 5, 1, 0, 6, 8, 2, 5, 6, 5,
        8, 2, 6, 5, 5, 5, 1, 8, 8, 0, 0, 6, 2, 8, 3, 0, 2, 1, 9, 6, 8, 7, 8, 6,
        7, 9, 8, 7, 2, 0, 9, 9, 2, 7, 5, 0, 8, 6, 6, 2, 3, 8, 6, 6, 3, 0, 5, 3,
        6, 1, 6, 4, 3, 2, 3, 3, 3, 8, 0, 7, 9, 3, 0, 2, 6, 8, 6, 9, 3, 9, 6, 3,
        0, 9, 2, 2, 2, 6, 8, 5, 3, 1, 6, 2, 2, 3, 3, 9, 3, 0, 8, 0, 2, 8, 6, 6,
        1, 3, 7, 8, 0, 6, 6, 8, 5, 8, 2, 3, 6, 1, 2, 0, 6, 8, 9, 6, 3, 9, 3, 3,
        0, 8, 3, 4, 7, 3, 2, 2, 6, 3, 8, 3, 0, 6, 2, 5, 8, 0, 3, 0, 8, 9, 0, 0,
        7, 0, 1, 2, 6, 8, 3, 2, 6, 0, 0,

In [ ]:
# 제출
comp.submit()

[Small Image Classification 평가 결과]
 AI Essential 1017(1) 과정 Hello World님의 점수는 0.382 입니다.